# Quantum Error Correction for Dummies (QEC4D)
# Chapter IV - Stabilisers

## 0) Overview
The stabilisers of a state are a group of operators which, when applied to said state, leave it unchanged. The idea behind this is that we can therefore define a state or space entirely based on its stabilisers.

### 0-A) A geometric intuition for the stabilisers' interest
For an intuition on how and why we would describe a space based on its stabilisers, let's look at a simple example involving basic geometrical shapes.
Imagine we were to encxode information using geometric shapes, and errors would be deformations causing a shape to transform into another.

First, we would pick 2 shapes to be used as our basic symbols for encoding. For our example, we'll pick squares and octagons.
Now let's consider that there are 2 types of errors, one that deforms a square or an octagon into an equilateral triangle and one that deforms them into an hexagon.

Now let's imagine that I create a symbol (square or octagon) and stick it into a black box without telling you what they are. From there, you are supposed to discover what the word is only by making tests that would leave the correct word unchanged. Sounds hard? There's 2 tricks to it. The questions you want to ask must:
1. Have a "yes" answer for both correct symbols (square, octagon) but a "no" answer for at least one of the incorrect symbols.
2. You want to ask questions to which one of the errors would answer "yes" and the other would answer "no".


#### Finding the right questions to ask
What questions do we have that have a *yes* answer for correct symbols? In other words, which geometrical properties are shared by both, the square and the octagon?
- Rotation by 90° gives back the same shape.
- Reflexion along the horizontal axis gives back the same shape.
- Reflexion along the vertical axis gives back the same shape.
All of these are true for both, the square and the octagon. We say that these properties ***stabilise*** the square and the octagon.

Now how about the errors? Let's agree that we set our shapes with the a flat side as base.
- *Rotation by 90° gives back the same shape.* This is false for the equilateral triangle and for the hexagon. We say that this propertie ***does not stabilise*** the equilateral triangle or the hexagon.
- *Reflexion along the horizontal axis gives back the same shape.* This is false for the equilateral triangle but true for the hexagon. This property ***stabilises*** the hexagon but ***does not stabilise*** the equilateral triangle.
- *Reflexion along the vertical axis gives back the same shape.* This is true for both the equilateral triangle and the hexagon, they are both ***stabilised*** by this property.

#### Asking the questions
I now claim that by asking sequentially 2 questions, I am completely able to determine which error has occured. How?

1. If I reflect along the horizontal axis, do I get back the same shape?
I will here get a *yes* answer for any shape except the equilateral triangle. If I get a *no* here, I know I have the equilateral triangle and that's it. But if I don't, it can either be no error or the error that turns shapes into hexagons. Therefore I need to ask one more question.
2. If I rotate the shape clock-wise by 90°, do I get back the same shape? If the answer is yes, it means that no error occurred. If the answer is no, I need to remember the answer to the first question. If it was "yes", then the error that occurred is an hexagon transformnation. If it was "no", then it is indeed the equilateral triangle.

Note that here, I have obtained all the information I needed only by asking questions that leave the correct symbols untouched. This is the core idea behind *stabilisers*.


## 1) The importance of commutation
One last property that stabilisers must have, is that they must ***commute*** with each other. In simple terms, this means that if I have 2 stabilisers A and B, applying A first then B should give me the same result as applying B first then A.
To keep our example with geometrical shapes, this means that :
1. First reflecting along the horizontal axis,
2. then rotating the shape clock-wise by 90°
Must give the same result on our shapes of interest (square and octagon) as
1. First rotating the shape by 90° clock-wise
2. then reflecting along the horizontal axis

Luckily this is true for our example.

But what about other shapes? This can't always be true!
Correct! And it doesn't need to be. Stabilisers are defined for a particular *thing* (called the ***code subspace***). It means that if I have my stabiliser for my specific *thing* called $\mathcal{C}$, then they must always commute for any element of $\mathcal{C}$. But I don't need to guarantee that this is true for elements outside of $\mathcal{C}$! So, can you find elements outside of your code subspace $\mathcal{C}$ on which stabilisers don't commute? Absolutely. And that's ok. For instance a rhombus or a parallelogram would be examples of shapes on which our stabiliser doesn't commute. But that's perfectly fine since they are not part of our code subspace.

In [2]:
# First, a few imports
import numpy as np
from copy import deepcopy
from numpy.random import rand, seed
from qiskit import *
from qiskit import Aer
from typing import List, Optional, Tuple

In [3]:
def simulate_measurements(circuit:QuantumCircuit, nb_shots:int=1024) -> dict:
    """Simulates measurement results."""
    backend_sim = Aer.get_backend('qasm_simulator')
    job_sim = backend_sim.run(transpile(circuit, backend_sim), shots=nb_shots)
    result_sim = job_sim.result()
    counts = result_sim.get_counts(circuit)
    return counts

def most_common_output(counts:dict) -> str:
    """ Provides the (q/c)-bit 0>n encoding of the most measured output."""
    return int(max(counts, key=counts.get)[::-1])

## 1) Examples of bit-value parity-check stabiliser elements

The eigenstates of the stabilisers are either $+1$ (for the code space) or $-1$ (for the error space).
To obtain those, one obtains the measurement outcomes (0s and 1s) and converts them according to a rule based on the encoding.
In our case: $\ket{0} \mapsto +1$ and $\ket{1} \mapsto -1$.

In [4]:
def convert_comp_basis_to_eigvals(result_str):
    first_elt = int(result_str[0])
    second_elt = int(result_str[1])
    first_eigval = to_eigval(first_elt)
    second_eigval = to_eigval(second_elt)
    return first_eigval * second_eigval

def to_eigval(x):
    return 1 if (x==0) else -1

### 1-A) The $\mathbf{ZZ}\mathbb{I}$ circuit

When talking about the $\mathbf{ZZ}\mathbb{I}$ stabiliser, the corresponding circuit is understood to:
- Be made of 3 qubits.
- Contain 2 control nots for: 1) the first and last qubit pair, 2) the middle and last qubit pair.
- Have Z-measurements (the computational basis ones) on the first and middle qubits.

The action of this stabiliser circuit is to perform a parity check on the bit value of the first and the second qubits. Concretely, it evaluates whether the first qubit is $\ket{0}$ or $\ket{1}$, then it looks at whether the second qubit is in state $\ket{0}$ or $\ket{1}$ and outputs an eigenvalue of $-1$ if they have different values, and an eigenvalue of $+1$ of they share the same value.

In [15]:
for i in range(3):    
    error_index = i
    zzi_circ = QuantumCircuit(QuantumRegister(3), ClassicalRegister(2))
    zzi_circ.x(error_index)
    zzi_circ.cx(0,2)
    zzi_circ.cx(1,2)
    zzi_circ.measure(0,0)
    zzi_circ.measure(1,1)
    backend_sim = Aer.get_backend('qasm_simulator')
    job_sim = backend_sim.run(transpile(zzi_circ, backend_sim), shots=1024)
    result_sim = job_sim.result()
    counts = result_sim.get_counts(zzi_circ)
    result = max(counts)
    resulting_eigval = convert_comp_basis_to_eigvals(result)
    print(f"With an error on qubit {error_index}, the syndrome was: {result} with an eigenvalue of {resulting_eigval}")


With an error on qubit 0, the syndrome was: 01 with an eigenvalue of -1
With an error on qubit 1, the syndrome was: 10 with an eigenvalue of -1
With an error on qubit 2, the syndrome was: 00 with an eigenvalue of 1


In [9]:
zzi_circ.draw('text')

┌─┐   
q8_0: ───────■───────┤M├───
             │       └╥┘┌─┐
q8_1: ───────┼────■───╫─┤M├
      ┌───┐┌─┴─┐┌─┴─┐ ║ └╥┘
q8_2: ┤ X ├┤ X ├┤ X ├─╫──╫─
      └───┘└───┘└───┘ ║  ║ 
c2: 2/════════════════╩══╩═
                      0  1

The question the $\mathbf{Z}\mathbf{Z}\mathbb{Z}$ stabiliser answers is: are the bit values of qubits 0 and 1 the same? A positive (true) answer will result in a $+1$ eigenvalue while a negative (false) answer will result in $-1$.

The truth table of the $\mathbf{Z}\mathbf{Z}\mathbb{I}$ stabiliser eigenvalue is as follows:

|     | $\mathbf{Z}\mathbf{Z}\mathbb{I}$ eigenvalue |
|-----|----------------------------------|
| 000 | +1 (True)                               |
| 001 | +1 (True)                             |
| 010 | -1 (False)                              |
| 011 | -1 (False)                              |
| 100 | -1 (False)                              |
| 101 | -1 (False)                              |
| 110 | +1 (True)                              |
| 111 | +1 (True)                              |

Paying close attention, you will notice that some bit strings that only differ by the value of the last qubit have different truth values. 

|     | $\mathbf{Z}\mathbf{Z}\mathbb{I}$ eigenvalue |
|-----|----------------------------------|
| 010 | -1 (False)                              |
| 011 | -1 (False)                              |

This pattern where strings that differ only by the value of a single bit can have different truth values means that the bit is irrelevant in the truth value. Think of it that way, if you love dogs and hate cats, if I tell you I have a brown dog you'll be happy to meet them, if I tell you I have a grey dog you'll also be happy. But if I tell you I have a grey cat you'll be as unhappy to meet it as if I had a brown cat. This is because the colour of the pet here is irrelevant, all you care about is the species. Knowing that, i know I can safely ignore the colour of the pet and still give you enough information that you'll know whether you're going to be relaxed or stressed when coming over for lunch.
This means that we can simplify the truth table to ignore the value of the last bit and still have as much information as before.

|     | $\mathbf{Z}\mathbf{Z}\mathbb{I}$ eigenvalue |
|-----|----------------------------------|
| 000 or 001 | +1 (True)                               |
| 010 or 011 | -1 (False)                              |
| 100 or 101 | -1 (False)                              |
| 110 or 111 | +1 (True)                              |

We could even condensate it further as follows:

|     | $\mathbf{Z}\mathbf{Z}\mathbb{I}$ eigenvalue |
|-----|----------------------------------|
| 000 or 001 or 110 or 111 | +1 (True)                               |
| 010 or 011 or 100 or 101| -1 (False)                              |

But wait... there's a problem now!
We accept that $\mathbf{Z}\mathbf{Z}\mathbb{I}$ can't handle the information given by the last qubit, so if the error occurs there (as in $001$ for instance) it will not be detected. Sure. But there's another problem now. If we started with $000$, the bitstrings $110$ and $111$ are also errors. But they give the answer: true! How can that be?
Remember that we mentionned that this code could only detect **a single** bit flip error? Well that's where it comes from. The only thing $\mathbf{Z}\mathbf{Z}\mathbb{I}$ can telle is whether qubits 0 and 1 have the same bit value or not. Nothing more. So while they will detect a single error such as $010$ or $011$ they will not pick up a double error $110$ or $111$ because, well... from the $\mathbf{Z}\mathbf{Z}\mathbb{I}$ stabiliser's perspective this is not *actually* an error... $111$ and $110$ are perfectly valid cases of both qubits 0 and 1 having the same value (here 1), so the answer is still true.

Still confused? Think about it that way. We *use* stabilisers like $\mathbf{Z}\mathbf{Z}\mathbb{I}$ to ask questions and based on those questions, we try to conclude what happened. But stabilisers themselves don't have a concept of a correct/incorrect message. they only answer questions. We must **interpret** those answers to know what happened. 

### 1-B) The $\mathbb{I}\mathbf{Z}\mathbf{Z}$ circuit
With the $\mathbb{I}\mathbf{Z}\mathbf{Z}$ stabiliser element, we tested the bit parity of both the first and second qubits. Now we want to test the parity of the last two qubits.

In [17]:
for i in range(3):    
    error_index = i
    izz_circ = QuantumCircuit(QuantumRegister(3), ClassicalRegister(2))
    izz_circ.x(error_index)
    izz_circ.cx(1,0)
    izz_circ.cx(2,0)
    izz_circ.measure(1,0)
    izz_circ.measure(2,1)
    backend_sim = Aer.get_backend('qasm_simulator')
    job_sim = backend_sim.run(transpile(izz_circ, backend_sim), shots=1024)
    result_sim = job_sim.result()
    counts = result_sim.get_counts(izz_circ)
    result = max(counts)
    resulting_eigval = convert_comp_basis_to_eigvals(result)
    print(f"With an error on qubit {error_index}, the syndrome was: {result} with an eigenvalue of {resulting_eigval}")


With an error on qubit 0, the syndrome was: 00 with an eigenvalue of 1
With an error on qubit 1, the syndrome was: 01 with an eigenvalue of -1
With an error on qubit 2, the syndrome was: 10 with an eigenvalue of -1


In [11]:
izz_circ.draw('text')

┌───┐┌───┐   ┌─┐
q34_0: ┤ X ├┤ X ├───┤M├
       └─┬─┘└─┬─┘┌─┐└╥┘
q34_1: ──■────┼──┤M├─╫─
       ┌───┐  │  └╥┘ ║ 
q34_2: ┤ X ├──■───╫──╫─
       └───┘      ║  ║ 
c10: 2/═══════════╩══╩═
                  1  0

### 1-C) Is that enough?

So far we can check the parity of the first two qubits and separately we can check the parity of the last two qubits. Do we need more? What about the parity of the first and last qubits? As you might have noticed, we don't need this value 